In [1]:
!pip3 install openmim
!mim install mmengine

# !rm -rf mmsegmentation
# !git clone https://github.com/open-mmlab/mmsegmentation.git 
# %cd mmsegmentation
# !pip install -e .


Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.1.0/index.html


In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmseg
print(mmseg.__version__)

import mmcv
import mmengine
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os.path as osp
import numpy as np
from PIL import Image

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = '1'

/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


2.1.0.post300 True
1.2.1


In [3]:
torch.cuda.is_available() == True

True

In [4]:
root = '/workspaces/ECE661GroupProject_TransferLearning/data'
img_dir = 'image_png_256'
ann_dir = 'fluid_png_256'

classes = ('image', 'fluid')


@DATASETS.register_module()
class BOE_Chiu_Dataset(BaseSegDataset):
    METAINFO = dict(classes = classes)
    def __init__(self,dataset=None, times=None, **kwargs):
        super(BOE_Chiu_Dataset, self).__init__(img_suffix='.png', seg_map_suffix='.png', **kwargs)


In [5]:
#from mmcv import Config
cfg = mmengine.Config.fromfile('/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/configs/unet/unet-s5-d16_fcn_4xb4-160k_cityscapes-512x1024.py')

In [6]:
# !mkdir checkpoints
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P checkpoints

In [7]:
# from mmseg.apis import set_random_seed
# from mmseg.utils import get_device

# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 2
cfg.model.auxiliary_head.num_classes = 2

# Modify dataset type and path
cfg.dataset_type = 'BOE_Chiu_Dataset'
cfg.data_root = root

# cfg.data.samples_per_gpu = 8
# cfg.data.workers_per_gpu=8
# cfg.crop_size = (256, 256)

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', scale=(512, 512), keep_ratio=True),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(512, 512), keep_ratio=True),
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]



cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = '/workspaces/ECE661GroupProject_TransferLearning/splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = '/workspaces/ECE661GroupProject_TransferLearning/splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = 200
cfg.train_cfg.val_interval = 200
cfg.default_hooks.logger.interval = 10
cfg.default_hooks.checkpoint.interval = 200

# Set seed to facitate reproducing the result
cfg['randomness'] = dict(seed=0)


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
crop_size = (
    512,
    1024,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        1024,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = '/workspaces/ECE661GroupProject_TransferLearning/data'
dataset_type = 'BOE_Chiu_Dataset'
default_hooks = dict(
    checkpoint=dict(by_epoch=False, interval=200, type='CheckpointHook'),
    logger=dict(interval=10, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook'))
default_scope = 'mmseg'
env_cfg = dict(
    cudnn_benchmark=True,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
img_ratios = [
    0.5,
    0.75,


In [8]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

12/10 21:12:09 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 | packaged by conda-forge | (default, Oct 10 2023, 15:44:36) [GCC 12.3.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla V100-PCIE-16GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.1.0.post300
    PyTorch compiling details: PyTorch built with:
  - GCC 11.4
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_35,code=sm_35;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode

/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:249: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


12/10 21:12:10 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
12/10 21:12:10 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/workspaces/ECE661GroupProject_TransferLearning/mmsegmentation/mmseg/engine/hooks/visualization_hook.py:61: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [9]:
runner.train()

12/10 21:12:11 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.


Traceback (most recent call last):
  File "/opt/conda/envs/openmmlab/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/opt/conda/envs/openmmlab/lib/python3.8/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torch/multiprocessing/reductions.py", line 428, in reduce_storage
    fd, size = storage._share_fd_cpu_()
  File "/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torch/storage.py", line 297, in wrapper
    return fn(self, *args, **kwargs)
  File "/opt/conda/envs/openmmlab/lib/python3.8/site-packages/torch/storage.py", line 330, in _share_fd_cpu_
    return super()._share_fd_cpu_(*args, **kwargs)
RuntimeError: unable to write to file </torch_18729_1521458785_7>: No space left on device (28)
Traceback (most recent call last):
  File "/opt/conda/envs/openmmlab/lib/python3.8/multiprocessing/queues.py", line 239, in _feed
    obj = 

SyntaxError: Failed to format the config file, please check the syntax of: 
crop_size=(
    512,
    1024,
    )
data_preprocessor=dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
        ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        1024,
        ),
    std=[
        58.395,
        57.12,
        57.375,
        ],
    type='SegDataPreProcessor')
data_root='/workspaces/ECE661GroupProject_TransferLearning/data'
dataset_type='BOE_Chiu_Dataset'
default_hooks=dict(
    checkpoint=dict(
        by_epoch=False,
        interval=200,
        type='CheckpointHook'),
    logger=dict(
        interval=10,
        log_metric_by_epoch=False,
        type='LoggerHook'),
    param_scheduler=dict(
        type='ParamSchedulerHook'),
    sampler_seed=dict(
        type='DistSamplerSeedHook'),
    timer=dict(
        type='IterTimerHook'),
    visualization=dict(
        type='SegVisualizationHook'))
default_scope='mmseg'
env_cfg=dict(
  